## 井点数据预处理


In [1]:
import numpy as np
import pandas as pd

In [2]:
file_well = "data/well_old.xlsx"
data_well = pd.read_excel(file_well, sheet_name="Sheet1")

data_well.head()

,X,Y,Z,TWT picked,TWT auto,Geological age,MD,PVD auto,Type,Surface,...,Geometric mean,Half energy,Harmonic mean,Maximum amplitude,Mean amplitude,Minimum amplitude,Most of,RMS amplitude,Sum of amplitudes,Sum of energy
0,686332.8,3217090.3,-2431.9,-999.0,-2054.65,-999.0,2505.70,-2431.9,Horizon,H4-1,...,-999.0,5.5,-4037.4,-1088.3,-7374.9,-11001.3,-11368.4,8358.9,-48991.8,476109952.0
1,686332.3,3217085.0,-2448.4,-999.0,-2063.95,-999.0,2523.03,-2448.4,Horizon,H4-2,...,-999.0,5.4,-4101.5,-1148.5,-7515.7,-10943.6,-11681.2,8564.2,-49956.6,491802848.0
2,686331.7,3217078.2,-2469.8,-999.0,-2076.06,-999.0,2545.50,-2469.8,Horizon,H4-3,...,-999.0,5.4,-4317.3,-1318.2,-7555.7,-10869.0,-11606.4,8573.0,-51396.2,504725728.0
3,686330.9,3217070.0,-2495.4,-999.0,-2090.63,-999.0,2572.41,-2495.4,Horizon,H4-4,...,-999.0,5.4,-4572.7,-1520.1,-7602.0,-10775.9,-11516.0,8582.5,-53099.7,519984608.0
4,686329.5,3217057.2,-2535.6,-999.0,-2113.26,-999.0,2614.60,-2535.6,Horizon,H5-1,...,-999.0,5.3,-4963.1,-1832.5,-7668.6,-11663.7,-11369.6,8592.8,-55698.0,543105152.0


In [3]:
# 只保留excel的A~C, J, K, AB~AR列
# 获取所有列名
all_columns = data_well.columns
# 选择需要保留的列
selected_columns = list(all_columns[0:3])  # A~C列
selected_columns.extend(list(all_columns[9:11]))  # J, K列
selected_columns.extend(list(all_columns[27:44]))  # AB~AR列

# 只保留需要的列，并删除层位是P0的行
filtered_data = data_well[selected_columns]
filtered_data = filtered_data[filtered_data["Surface"] != "P0"]
print("筛选后的列：", filtered_data.columns.tolist())
print("筛选后的数据形状：", filtered_data.shape)
print("\n筛选后的数据前5行：")
print(filtered_data.head())

筛选后的列： ['X', 'Y', 'Z', 'Surface', 'Well', 'Thickness of facies(1: Fine sand)', 'facies(1: Fine sand)', 'Average energy', 'Average envelope', 'Average instantaneous frequency', 'Average instantaneous phase', 'Average peak value', 'Geometric mean', 'Half energy', 'Harmonic mean', 'Maximum amplitude', 'Mean amplitude', 'Minimum amplitude', 'Most of', 'RMS amplitude', 'Sum of amplitudes', 'Sum of energy']
筛选后的数据形状： (1084, 22)

筛选后的数据前5行：
          X          Y       Z Surface Well  \
0  686332.8  3217090.3 -2431.9    H4-1   A1   
1  686332.3  3217085.0 -2448.4    H4-2   A1   
2  686331.7  3217078.2 -2469.8    H4-3   A1   
3  686330.9  3217070.0 -2495.4    H4-4   A1   
4  686329.5  3217057.2 -2535.6    H5-1   A1   

   Thickness of facies(1: Fine sand) facies(1: Fine sand) Average energy  \
0                              14.41                87.31       71382104   
1                              14.16                66.18       73817448   
2                               4.19               

In [4]:
# 统计"Well"这一列有多少个值，并处理每个Well
well_column = filtered_data.columns[4]  # 井名在第4列
sand_thickness_column = filtered_data.columns[5]  # 砂厚在第5列
sand_ratio_column = filtered_data.columns[6]  # 砂地比在第6列

unique_wells = filtered_data[well_column].unique()
print(f"\n共有 {len(unique_wells)} 个不同的井点值")

# 存储处理后的数据
final_data = pd.DataFrame(columns=filtered_data.columns)

# 要删除的井点列表
wells_to_delete = []

# 循环处理每个井点
for well in unique_wells:
    # 1. 删除指定井点
    if well in wells_to_delete:
        print(f"\n井点 {well} 在指定删除列表中，已删除。")
        continue

    # 获取当前井的数据
    well_data = filtered_data[filtered_data[well_column] == well]

    # 2. 删除少于10行的井点
    if len(well_data) < 10:
        print(f"\n井点 {well} 只有 {len(well_data)} 行数据，已删除。")
        continue

    # 3. 检查砂厚和砂地比列的异常值比例
    sand_thickness_anomaly_ratio = (well_data[sand_thickness_column] == -999).mean()
    sand_ratio_anomaly_ratio = (well_data[sand_ratio_column] == -999).mean()

    # 如果砂厚或砂地比的异常值比例超过70%，则删除该井
    if sand_thickness_anomaly_ratio > 0.7 or sand_ratio_anomaly_ratio > 0.7:
        print(
            f"\n井点 {well} 的砂厚异常值比例为 {sand_thickness_anomaly_ratio:.2%}，砂地比异常值比例为 {sand_ratio_anomaly_ratio:.2%}，已删除。"
        )
        continue

    # 保留符合条件的井点数据
    final_data = pd.concat([final_data, well_data], ignore_index=True)
    print(f"\n井点 {well} 有 {len(well_data)} 行数据，已保留。前5行数据：")
    print(well_data.head())

print(f"\n处理后总共保留了 {len(final_data)} 行数据，包含 {len(final_data[well_column].unique())} 个井点")


共有 41 个不同的井点值

井点 A1 有 28 行数据，已保留。前5行数据：
          X          Y       Z Surface Well  \
0  686332.8  3217090.3 -2431.9    H4-1   A1   
1  686332.3  3217085.0 -2448.4    H4-2   A1   
2  686331.7  3217078.2 -2469.8    H4-3   A1   
3  686330.9  3217070.0 -2495.4    H4-4   A1   
4  686329.5  3217057.2 -2535.6    H5-1   A1   

   Thickness of facies(1: Fine sand) facies(1: Fine sand) Average energy  \
0                              14.41                87.31       71382104   
1                              14.16                66.18       73817448   
2                               4.19                16.35       74010816   
3                              18.36                45.73       74225072   
4                               0.00                    0       74485976   

   Average envelope  Average instantaneous frequency  ...  Geometric mean  \
0            9259.5                             27.8  ...          -999.0   
1            9455.2                             27.7  ...       

C:\Users\WangQinZhuo\AppData\Local\Temp\ipykernel_21824\1104142178.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat([final_data, well_data], ignore_index=True)


In [5]:
# 删除异常值(-999)数量大于60%的列
# 计算每列中-999的比例
anomaly_ratio = {}
for col in final_data.columns:
    if final_data[col].dtype in [np.int64, np.float64]:  # 只检查数值列
        ratio = (final_data[col] == -999).mean()
        anomaly_ratio[col] = ratio
        print(f"列 {col} 中异常值(-999)的比例为: {ratio:.2%}")

# 筛选出异常值比例不超过60%的列
good_columns = [col for col in final_data.columns if col not in anomaly_ratio or anomaly_ratio[col] <= 0.6]
clean_data = final_data[good_columns]

print(f"\n删除异常值比例大于60%的列后，剩余 {len(good_columns)} 列")
print("最终保留的列：", good_columns)
print("\n最终数据的前5行：")
print(clean_data.head())

# 保存处理后的数据
clean_data.to_excel("data/well_processed.xlsx", index=False)
print("\n处理后的数据已保存到 'data/well_processed.xlsx'")

列 Y 中异常值(-999)的比例为: 0.00%
列 Z 中异常值(-999)的比例为: 0.00%
列 Thickness of facies(1: Fine sand) 中异常值(-999)的比例为: 1.70%
列 Average envelope 中异常值(-999)的比例为: 0.00%
列 Average instantaneous frequency 中异常值(-999)的比例为: 0.00%
列 Average instantaneous phase 中异常值(-999)的比例为: 0.00%
列 Average peak value 中异常值(-999)的比例为: 68.90%
列 Geometric mean 中异常值(-999)的比例为: 90.98%
列 Half energy 中异常值(-999)的比例为: 0.00%
列 Harmonic mean 中异常值(-999)的比例为: 0.00%
列 Maximum amplitude 中异常值(-999)的比例为: 0.00%
列 Mean amplitude 中异常值(-999)的比例为: 0.00%
列 Minimum amplitude 中异常值(-999)的比例为: 0.00%
列 Most of 中异常值(-999)的比例为: 0.00%
列 RMS amplitude 中异常值(-999)的比例为: 0.00%
列 Sum of amplitudes 中异常值(-999)的比例为: 0.00%
列 Sum of energy 中异常值(-999)的比例为: 0.00%

删除异常值比例大于60%的列后，剩余 20 列
最终保留的列： ['X', 'Y', 'Z', 'Surface', 'Well', 'Thickness of facies(1: Fine sand)', 'facies(1: Fine sand)', 'Average energy', 'Average envelope', 'Average instantaneous frequency', 'Average instantaneous phase', 'Half energy', 'Harmonic mean', 'Maximum amplitude', 'Mean amplitude', 'Minim